# Redes Neuronales Recurrentes (RNN) en Procesamiento de Lenguaje Natural (NLP)

En este cuaderno, implementaremos y experimentaremos con redes neuronales recurrentes (RNN) y sus variantes, como las LSTM y GRU, aplicándolas a tareas comunes de procesamiento de lenguaje natural (NLP).

## Requisitos previos
- Python 3.7+
- Numpy
- TensorFlow

## Objetivo
1. Comprender la arquitectura y funcionamiento de las RNN y sus variantes.
2. Implementar RNN, LSTM y GRU aplicadas a tareas de clasificación de texto.
3. Evaluar y comparar el rendimiento de cada modelo.

## 1. Cargando y Preprocesando el Dataset
En esta sección, cargaremos un dataset de texto para realizar clasificación. Utilizaremos técnicas de preprocesamiento como la tokenización y el padding para preparar los datos para los modelos.


In [4]:
### Leemos el documento en txt 

import requests
import re
import os

url = "https://gist.githubusercontent.com/ismaproco/6781d297ee65c6a707cd3c901e87ec56/raw/20d3520cd7c53d99215845375b1dca16ac827bd7/gabriel_garcia_marquez_cien_annos_soledad.txt"

response = requests.get(url)

if response.status_code == 200:
    with open("gabriel_garcia_marquez_cien_annos_soledad.txt", "w") as file:
        file.write(response.text)
else:
    print("Error al descargar el archivo")

# Leemos el archivo

with open("gabriel_garcia_marquez_cien_annos_soledad.txt", "r") as file:
    texto = file.read()

# Arreglamos el texto


texto = texto.replace("\n", " ")
texto = texto.replace("\r", " ")
texto = texto.replace("Gabriel García Márquez", " ")
texto = texto.replace("Cien años de soledad", " ")
texto = texto.replace("  ", " ")

texto = re.sub(r'\s+', ' ', texto)

texto = texto.replace('EDITADO POR "EDICIONES LA CUEVA" Para J omi García Ascot y María Luisa Elio ', " ")

## Guardamos el texto arreglado

with open("gabriel_garcia_marquez_cien_annos_soledad.txt", "w") as file:
    file.write(texto)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd

# Cargar el dataset

data = pd.read_csv('path_to_dataset.csv')  # Reemplaza con el dataset adecuado

# Preprocesamiento del texto
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(data['text_column'])
sequences = tokenizer.texts_to_sequences(data['text_column'])

# Padding para tener secuencias de igual longitud
padded_sequences = pad_sequences(sequences, padding='post', maxlen=100)
padded_sequences[:5]  # Muestra las primeras 5 secuencias procesadas

## 2. Implementando una Red Neuronal Recurrente (RNN) Simple
Ahora implementaremos una RNN simple usando TensorFlow. Este modelo será nuestro punto de partida para comprender cómo funciona una RNN aplicada a NLP.

In [ ]:
# Crear una red RNN simple
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64, input_length=100),
    tf.keras.layers.SimpleRNN(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history = model.fit(padded_sequences, data['label_column'], epochs=5, validation_split=0.2)

## 3. Implementando LSTM
A continuación, reemplazaremos la capa SimpleRNN con una capa LSTM y entrenaremos el modelo de nuevo. Esto nos permitirá comparar el rendimiento de ambos enfoques.

In [ ]:
# Crear una red LSTM
model_lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64, input_length=100),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history_lstm = model_lstm.fit(padded_sequences, data['label_column'], epochs=5, validation_split=0.2)

## 4. Evaluación del Modelo
En esta sección, evaluaremos ambos modelos (RNN y LSTM) y visualizaremos su rendimiento durante el entrenamiento usando gráficos.

In [ ]:
import matplotlib.pyplot as plt

# Visualizar la pérdida
plt.plot(history.history['loss'], label='RNN')
plt.plot(history_lstm.history['loss'], label='LSTM')
plt.title('Pérdida del modelo')
plt.xlabel('Épocas')
plt.ylabel('Pérdida')
plt.legend()
plt.show()

# Evaluar ambos modelos
rnn_eval = model.evaluate(padded_sequences, data['label_column'])
lstm_eval = model_lstm.evaluate(padded_sequences, data['label_column'])
print(f"Evaluación RNN: {rnn_eval}")
print(f"Evaluación LSTM: {lstm_eval}")

## 5. Implementando GRU
Finalmente, implementaremos una red GRU. Las GRU son una variante simplificada de las LSTM, y compararemos su rendimiento con las RNN y LSTM.

In [ ]:
# Crear una red GRU
model_gru = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=5000, output_dim=64, input_length=100),
    tf.keras.layers.GRU(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compilar el modelo
model_gru.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entrenar el modelo
history_gru = model_gru.fit(padded_sequences, data['label_column'], epochs=5, validation_split=0.2)

## 6. Conclusión y Reflexión
Reflexiona sobre las ventajas y desventajas de cada tipo de red recurrente. ¿Qué tipo de red fue más eficaz en este dataset? ¿Qué diferencias observaste entre la RNN, LSTM y GRU en términos de rendimiento y tiempo de entrenamiento?